<a href="https://colab.research.google.com/github/khietvuarong/Coding-Exercise---Prompt-Engineering/blob/main/MinhKhietLe_Code_Generation_with_ReACT_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================================
# ReACT Code Generation System (Colab Secret Version)
# Secure API Key Handling
# ==========================================================

!pip install -U google-genai

In [38]:
# ==========================================================
# CLEAN ReACT Code Generation System (Colab Secret Only)
# ==========================================================

USE_API = True
MAX_ITERATIONS = 5

import subprocess
import sys
import traceback

# ==========================================================
# LOAD SECRET KEY FROM COLAB
# ==========================================================

if USE_API:
    try:
        from google.colab import userdata
        from google import genai

        api_key = userdata.get('SecretKey')

        if not api_key:
            raise ValueError("SecretKey not found in Colab secrets.")

        client = genai.Client(api_key=api_key)

        # Auto-detect first available model
        models = [m.name.replace("models/", "") for m in client.models.list()]
        if not models:
            raise RuntimeError("No models available for this key.")

        MODEL_NAME = models[0]
        print("✅ Using model:", MODEL_NAME)

    except Exception as e:
        print("❌ API initialization failed:")
        print(e)
        print("⚠ Switching to fallback mode.")
        USE_API = False


# ==========================================================
# REACT PROMPT TEMPLATE
# ==========================================================

def build_prompt(problem, previous_error=None):

    prompt = f"""
You are a Python coding agent using ReACT methodology.

STEPS:
1. Generate ONLY valid Python code.
2. Include basic error handling.
3. Print final output clearly.
4. Do NOT include explanations or markdown.

Problem:
{problem}
"""

    if previous_error:
        prompt += f"\nThe previous execution produced this error:\n{previous_error}\nFix it."

    return prompt


# ==========================================================
# EXECUTE CODE
# ==========================================================

def execute_code(code):
    try:
        result = subprocess.run(
            [sys.executable, "-c", code],
            capture_output=True,
            text=True
        )
        if result.returncode == 0:
            return True, result.stdout
        else:
            return False, result.stderr
    except Exception:
        return False, traceback.format_exc()


# ==========================================================
# MAIN REACT LOOP
# ==========================================================

def react_code_generation(problem):

    print("\n=== Starting ReACT Cycle ===")

    previous_error = None
    code = None

    for iteration in range(1, MAX_ITERATIONS + 1):

        print(f"\n--- Iteration {iteration} ---")

        if USE_API:
            prompt = build_prompt(problem, previous_error)
            try:
                response = client.models.generate_content(
                    model=MODEL_NAME,
                    contents=prompt
                )
                code = response.text.strip()
            except Exception as e:
                print("❌ API call failed:")
                print(e)
                return
        else:
            # Simple fallback (static)
            code = """
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

print(is_prime(29))
"""

        print("\nGenerated Code:\n")
        print(code)

        success, output = execute_code(code)

        print("\nExecution Output:\n")
        print(output)

        if success:
            print("\n✅ Code executed successfully.")
            return

        print("\n❌ Error detected. Sending error back to model...")
        previous_error = output

    print("\n⚠ Max iterations reached without success.")


# ==========================================================
# RUN EXAMPLE
# ==========================================================

print("\nEnter your coding problem below:")
problem_statement = input(">> ")

react_code_generation(problem_statement)

✅ Using model: gemini-2.5-flash

Enter your coding problem below:
>> Create a python script that find the cube root of 49, round up to 2 decimals place

=== Starting ReACT Cycle ===

--- Iteration 1 ---

Generated Code:

import math

try:
    number = 49
    
    # Calculate the cube root
    cube_root = number**(1/3)
    
    # Round up to 2 decimal places
    # Multiply by 100, ceil, then divide by 100
    rounded_up = math.ceil(cube_root * 100) / 100.0
    
    print(f"{rounded_up}")

except Exception as e:
    print(f"An error occurred: {e}")

Execution Output:

3.66


✅ Code executed successfully.
